# Entrega L2 - Aprendizaje Bayesiano

### Grupo 38:
     - S. Robaina  C.I... 5.310.579-8
     - A. Baptista C.I... 4.883.322-3


##  1. Introducción

En esta tarea, nos enfocaremos en implementar y evaluar el rendimiento del algoritmo de clasificación conocido como "Naive Bayes" en el contexto de la clasificación de texto. El algoritmo de Naive Bayes nos permite construir un clasificador de texto utilizando un modelo relativamente simple.

Para lograr esto, entrenaremos un modelo de predicción de texto utilizando datos recopilados de conversaciones de WhatsApp. Durante este proceso, analizaremos cómo diferentes posibilidades de preprocesamiento y configuración de los parámetros afectan los resultados de nuestras pruebas. Evaluaremos la coherencia de las sugerencias generadas por el modelo y examinaremos las probabilidades con las que se asignan estas sugerencias.

En resumen, nuestro objetivo es examinar cómo el algoritmo de Naive Bayes se desempeña en la clasificación de texto, utilizando datos de WhatsApp, y cómo diversos factores, como el preprocesamiento y los parámetros, impactan en los resultados de nuestras pruebas.

## 2. Objetivo

*   Implementar el algoritmo Naive Bayes para desarrollar un modelo que permita predecir palabras.

*   Comparar su desempeño variando el preprocesamiento y los hiperparámetros horizonte(N) y m (m-estimador).

*   Evaluar su capacidad de aprendizaje reentrenando el modelo.

## 3. Diseño

### 3.1 Preprocesamiento de datos
*   Como el formato en el que se exportan los mensajes de _WhatsApp_ depende del sistema operativo los formatos en android e IOS son los siguientes:
    *   D/M/AA HH:MM - Nombre: (mensaje)
    *   [D/M/AA HH:MM] Nombre : (mensaje)
    
     Por este motivo se extrajeron los mensajes conservando únicamente el contenido posterior a ":" en cada línea. 
     
     Luego se generó una lista compuesta de las palabras de los mensajes conservados. 
     Una problemática que podría surgir de almacenar los datos de esta forma es que si tomamos todas las sub listas de largo _n_, algunas frases serán el resultado de concatenar el final de uno de los mensajes con el comienzo de otro. Entendemos que esto puede no generar problemas debido a que las frases que se generan en esos contextos son en principio aleatorias (dependiendo del caso concreto de los datos) o suficientemente específicas (por ejemplo en mensajes como chau, hola que marcan inicios o cierres) como para no influir en la generalidad del modelo. Teniendo en cuenta la dificultad agregada de almacenar los datos respetando los mensajes decidimos mantener este formato.

*   Al exportar los mensajes de _WhatsApp_ a un archivo de texto se incluyen algunas frases que no son mensajes como tal, por ejemplo: "Se eliminó este mensaje", "Multimedia omitido", etc. Evidenciamos que estas frases pueden tener un peso estadístico considerable, por lo que decidimos eliminarlas de los datos para no introducir un sesgo en el modelo generado.

*   Como el lenguaje en _WhatsApp_ no suele ser muy riguroso en términos de errores ortográficos y abreviaciones, consideramos que tenemos una importante fuente de ruido en nuestros datos. Para resolver este problema probamos cruzar las palabras con un diccionario en español (disponible en: https://github.com/words/an-array-of-spanish-words), descartando aquellas palabras no presentes en el mismo. En el loop principal se incluyó una variable booleana _Diccionario_ para que la predicción de la palabra se haga en base a los datos filtrados cuando vale True, y con todos los datos en otro caso.
El problema es que este diccionario no incluye palabras con tilde, por lo que elimina un gran número de palabras que no contenían errores ortográficos. Para lidiar con esta situación nos planteamos dos posibilidades:
    *   Que las palabras que no se encontraran en el diccionario fueran corregidas utilizando la biblioteca _pyspellchecker_. Esta opción se descarta debido a que el tiempo de procesamiento era demasiado elevado.
    
    *   Eliminar los tildes de las palabras previo a cruzarlas con el diccionario. Reconocemos que esta alternativa nos genera una limitante importante ya que la acentuación es de vital importancia en el idioma español. De todas formas se probó esta alternativa y se la comparó a los resultados utilizando datos sin corrección ortográfica, con el fin de experimentar y evaluar el ruido introducido por errores ortográficos.

### 3.2 Algoritmo

*   Las probabilidades utilizadas para determinar h_MAP se emplearon en su forma logarítmica, con el fin de evitar valores muy bajos de probabilidad que pueden disminuir la estabilidad numérica.

### 3.3 Evaluación
*   El desempeño de los distintos modelos generados se evaluó mediante el ingreso de 5 inicios de frases diferentes (por ejemplo: "buenas", "si yo opino que", etc.). Con estas frases de distintos largos buscamos simular escenarios de uso real teniendo palabras frecuentes tanto al inicio como en el desarrollo de una conversación, las frases son abiertas para permitir que el modelo pueda dar sugerencias potencialmente distintas. Se ingresan estas frases y se permite al modelo hacer 5 sugerencias de palabras luego. De las sugerencias realizadas por los modelos se evaluó: 

    *   Confiabilidad de la sugerencia: Se calcula el promedio de los factores de probabilidad de cada palabra sugerida. Esto permite que las métricas obtenidas para distintos valores de horizonte(N) sean más comparables, especialmente para valores de N más grandes. Para comparar el desempeño de la frase generada en su totalidad, se realizó un promedio de las confiabilidades de las palabras sugeridas (en las tablas de resultados aparecerá como "Resumen")

    *   Coherencia o sentido de la sugerencia: Se asignó un valor subjetivo del 1 al 5 que evaluará el nivel de coherencia de las frases generadas por los modelos al aceptar las 5 sugerencias. La coherencia será puntuada por el emisor de los mensajes, con el fin de no penalizar en exceso a los modelos. Le daremos especial valor esta métrica ya que entendemos que es la que nos indica con mayor certeza el buen funcionamiento de las sugerencias realizadas.

Los resultados se encuentran descritos en la tablas más abajo.

## 4. Experimentación

Para la experimentación se utilizó el chat con una persona y se contaban con 40.000 mensajes (máximo que permite exportar whatsapp), por problemas con el tiempo de procesamiento y entrenamiento se utilizó solamente la mitad de dicho archivo que de todos modos contenía 2 meses de conversaciones. Aún habiendo utilizado la mitad, los tiempos de procesamiento y entrenamiento de los modelos estuvieron entre los 5 y 10 minutos.

Observamos que la cantidad promedio de frases que quedan asociadas a cada palabra es cercano a 6, es decir que cada vez que calculemos P(palabra 1| palabra 2) en promedio la cantidad total de frases que terminan en palabra 2 será 6. Es por esto que utilizar un m para el m-estimador demasiado alto genera un incremento en la cantidad de stopwords, estas últimas tienen más frases asociadas debido a que aprecen mucho más en el texto y un m mayor las afecta menos que a aquellas palabras que tienen menos frases asociadas donde su probabilidad puede quedar casi como 1/|vocabulario|. Por esto realizamos pruebas con m = 4 y m = 2 y obtuvimos resultados muy similares en ambos casos, por esto solo incluimos aquellos resultados con m = 2
<br><br>

<font size=4>

_En la tabla 1 se presentan los resultados de confiabilidad obtenidos utilizando las palabras sin filtrar por diccionario_
</font>

<font size=5>Tabla 1</font>

<font size=2>

| m (m-estimador) | horizonte (N) | Frase (inicial) | Sugerencia 1 | Promedio 1 | Sugerencia 2 | Promedio 2 | Sugerencia 3 | Promedio 3 | Sugerencia 4 | Promedio 4 | Sugerencia 5 | Promedio 5 | Promedio final | Resumen |
|-----------------|---------------|-----------------|--------------|------------|--------------|------------|--------------|------------|--------------|------------|--------------|------------|----------------|---------------|
| 2               | 1             | buenas          | vibras       | 0,5        | si           | 0,015      | no           | 0,058      | me           | 0,15       | parece       | 0,65       | 0,2288333333   | 0,1931666667  |
|                 |               | si yo opino que | no           | 0,14       | me           | 0,15       | parece       | 0,65       | que          | 0,059      | no           | 0,14       | 0,1898333333   |               |
|                 |               | la ultima vez   | que          | 0,049      | no           | 0,14       | me           | 0,15       | parece       | 0,65       | que          | 0,059      | 0,1746666667   |               |
|                 |               | hola como andan | a            | 0,02       | la           | 0,095      | verdad       | 0,86       | que          | 0,063      | no           | 0,14       | 0,1963333333   |               |
|                 |               | saben si        | no           | 0,058      | me           | 0,15       | parece       | 0,65       | que          | 0,059      | no           | 0,14       | 0,1761666667   |               |
|                 |               |                 |              |            |              |            |              |            |              |            |              |            |                |               |
| 2               | 2             | buenas          | jajajaja     | 0,01       | descanso     | 0,065      | _nombre_       | 0,19       | nos          | 0,0068     | vemos        | 0,32       | 0,09863333333  | 0,107765      |
|                 |               | si yo opino que | estaría     | 0,13       | bueno        | 0,05       | bien         | 0,023      | pero         | 0,032      | ta           | 0,17       | 0,0675         |               |
|                 |               | la ultima vez   | había       | 0.10       | sido         | 0,13       | observación  | 0,25       | tuya         | 0,067      | ósea         | 0,0091     | 0,051525       |               |
|                 |               | hola como andan | yo          | 0,12       | a            | 0,17       | que          | 0,049      | hora         | 0,48       | te           | 0,12       | 0,1565         |               |
|                 |               | saben si        | te          | 0,39       | queres       | 0,25       | cuento       | 0,26       | cuando       | 0,011      | vaya         | 0,077      | 0,1646666667   |               |
|                 |               |                 |              |            |              |            |              |            |              |            |              |            |                |               |
| 2               | 3             | buenas          | jajajaja     | 0,01       | descanse     | 0,065      | _nombre_       | 0,046      | sticker      | 0,046      | buen         | 0,016      | 0,0305         | 0,04483333333 |
|                 |               | si yo opino que | estaría     | 0,12       | bien         | 0,052      | bueno        | 0,054      | conocer      | 0,14       | igual        | 0,032      | 0,06633333333  |               |
|                 |               | la ultima vez   | ayer         | 0,032      | había        | 0,011      | me           | 0,013      | con          | 0,04       | que          | 0,05       | 0,02433333333  |               |
|                 |               | hola como andan | que          | 0,021      | yo           | 0,07       | quw          | 0,2        | voy          | 0,085      | a            | 0,051      | 0,07116666667  |               |
|                 |               | saben si        | cero         | 0,068      | cosas        | 0,016      | pienso       | 0,083      | mm           | 0,011      | capaz        | 0,013      | 0,03183333333  |               |
|                 |               |                 |              |            |              |            |              |            |              |            |              |            |                |               |
| 2               | 4             | buenas          | jajajaja     | 0,01       | descanse     | 0,087      | _nombre_       | 0,057      | sticker      | 0,51       | buen         | 0,021      | 0,1141666667   | 0,1041133333  |
|                 |               | si yo opino que | estaría     | 0,12       | bien         | 0,057      | re           | 0,056      | bueno        | 0,061      | tener        | 0,041      | 0,05583333333  |               |
|                 |               | la ultima vez   | sido         | 0,038      | cuando       | 0,016      | cumplió      | 0,25       | 40           | 0,15       | lo           | 0,0064     | 0,07673333333  |               |
|                 |               | hola como andan | la           | 0,015      | que          | 0,37       | verdad       | 0,24       | es           | 0,088      | viaje        | 0,14       | 0,1421666667   |               |
|                 |               | saben si        | no           | 0,05       | sabía        | 0,24       | te           | 0,07       | contado      | 0,3        | iba          | 0,13       | 0,1316666667   |               |
</font>


<font size=4>
<br>

_En la tabla 2 se presentan los resultados de coherencia obtenidos utilizando las palabras sin filtrar por diccionario_
</font>

<font size=5>Tabla 2</font>

<font size=2>

| m (m-estimador) | horizonte (N) | Frase (final)                                | Coherencia | Coherencia total |
|-----------------|---------------|---------------------------------------------|------------|-------------------|
| 2               | 1             | buenas vibras si no me parece              | 3          | 13                |
|                 |               | si yo opino que no me parece que no       | 5          |                   |
|                 |               | la ultima vez que no me parece que        | 2          |                   |
|                 |               | hola como andan a la verdad que no        | 2          |                   |
|                 |               | saben si no me parece que no              | 1          |                   |
|                 |               |                                           |            |                   |
| 2               | 2             | buenas jajajaja descanse nombre nos vemos | 2          | 14                |
|                 |               | si yo opino que estaría bueno bien pero ta | 2        |                   |
|                 |               | la ultima vez había sido observación tuya ósea | 5     |                   |
|                 |               | hola como andan yo a que hora te          | 4          |                   |
|                 |               | saben si te queres cuento cuando vaya      | 1          |                   |
|                 |               |                                           |            |                   |
| 2               | 3             | buenas jajajaja descanse nombre sticker buen | 2        | 13                |
|                 |               | si yo opino que estaría bien bueno conocer igual | 5    |                   |
|                 |               | la ultima vez ayer había me con que       | 2          |                   |
|                 |               | hola como andan que yo quw voy a          | 1          |                   |
|                 |               | saben si cero cosas pienso mm capaz       | 3          |                   |
|                 |               |                                           |            |                   |
| 2               | 4             | buenas jajajaja descanse nombre sticker buen | 2        | 15                |
|                 |               | si yo opino que estaría buen re bueno tener | 3        |                   |
|                 |               | la ultima vez sido cuando cumplió 40 lo   | 3          |                   |
|                 |               | hola como andan la que verdad es viaje    | 5          |                   |
|                 |               | saben si no sabía te contado iba         | 2          |                   |
</font>

<font size = 4>
<br>

_En la tabla 3 se presentan los resultados de confiabilidad obtenidos utilizando las palabras filtradas por diccionario_
</font>

<font size=5>Tabla 3</font>

<font size = 2>

| m (m-estimador) | horizonte | Frase (inicial) | Sugerencia 1 | Promedio 1 | Suegerencia 2 | Promedio 2 | Suegerencia 3 | Promedio 3 | Suegerencia 4 | Promedio 4 | Suegerencia 5 | Promedio 5 | Promedio final | Resumen        |
|-----------------|-----------|-----------------|--------------|------------|--------------|------------|--------------|------------|--------------|------------|--------------|------------|----------------|----------------|
| 2               | 2         | buenas          | la           | 0,023      | es           | 0,038      | que          | 0,047      | no           | 0,062      | me           | 0,097      | 0,0445         | 0,04904666667  |
|                 |           | si yo opino que | estaria      | 0,092      | re           | 0,037      | alta         | 0,08       | bien         | 0,042      | si           | 0,2        | 0,07516666667  |                |
|                 |           | la ultima vez   | que          | 0,035      | habia        | 0,068      | hace         | 0,044      | un           | 0,035      | poco         | 0,022      | 0,034          |                |
|                 |           | hola como andan | que          | 0,024      | andas        | 0,17       | yo           | 0,048      | la           | 0,028      | estaba       | 0,077      | 0,05783333333  |                |
|                 |           | saben si        | te           | 0,045      | cero         | 0,038      | cosas        | 0,011      | quiero       | 0,0844     | que          | 0,024      | 0,03373333333  |                |
|                 |           |                 |              |            |              |            |              |            |              |            |              |            |                |                |
| 2               | 3         | buenas          | la           | 0,023      | verdad       | 0,044      | terrible      | 0,047      | es           | 0,035      | que          | 0,051      | 0,03333333333  | 0,06643333333  |
|                 |           | si yo opino que | estaría      | 0,12       | re           | 0,054      | bien         | 0,055      | bueno        | 0,056      | tener        | 0,039      | 0,054          |                |
|                 |           | la ultima vez   | que          | 0,044      | estabas      | 0,13       | en           | 0,041      | el           | 0,091      | no           | 0,083      | 0,06483333333  |                |
|                 |           | hola como andan | la           | 0,016      | que          | 0,037      | verdad       | 0,24       | es           | 0,084      | no           | 0,086      | 0,07716666667  |                |
|                 |           | saben si        | no           | 0,051      | sabia        | 0,25       | te           | 0,069      | puedo        | 0,17       | hacer        | 0,077      | 0,1028333333   |                |
|                 |           |                 |              |            |              |            |              |            |              |            |              |            |                |                |
| 2               | 4         | buenas          | que          | 0,055      | no           | 0,24       | se           | 0,23       | habia        | 0,16       | me           | 0,13       | 0,1358333333   | 0,09833333333  |
|                 |           | si yo opino que | estaría      | 0,11       | bien         | 0,069      | frenamos     | 0,18       | un           | 0,059      | que          | 0,032      | 0,075          |                |
|                 |           | la ultima vez   | que          | 0,052      | estabas      | 0,13       | fue          | 0,073      | pensando     | 0,063      | en           | 0,074      | 0,06533333333  |                |
|                 |           | hola como andan | que          | 0,019      | la           | 0,051      | verdad       | 0,25       | es           | 0,1        | no           | 0,11       | 0,08833333333  |                |
|                 |           | saben si        | no           | 0,061      | sabía        | 0,23       | te           | 0,082      | contado      | 0,25       | iba          | 0,14       | 0,1271666667   |                |
</font>

<font size = 4>
<br>

_En la tabla 4 se presentan los resultados de coherencia obtenidos utilizando las palabras filtradas por diccionario_
</font>

<font size=5>Tabla 4</font>

<font size = 2>

| m (m-estimador) | horizonte (N) | Frase (final)                                | Coherencia | Coherencia total |
|-----------------|---------------|---------------------------------------------|------------|-------------------|
| 2               | 2             | buenas la es que no me                     | 1          | 9                 |
|                 |               | si yo opino que  estaria re alta bien si  | 2          |                   |
|                 |               | la ultima vez que habia hace un poco      | 2          |                   |
|                 |               | hola como andan que andas yo la estaba    | 2          |                   |
|                 |               | saben si te cero cosas quiero que         | 2          |                   |
|                 |               |                                           |            |                   |
| 2               | 3             | buenas la verdad terrible es que           | 4          | 16                |
|                 |               | si yo opino que  estaría re bien bueno tener | 3        |                   |
|                 |               | la ultima vez que estabas en el no        | 2          |                   |
|                 |               | hola como andan la que verdad es no      | 4          |                   |
|                 |               | saben si no sabia te  puedo hacer         | 3          |                   |
|                 |               |                                           |            |                   |
| 2               | 4             | buenas que no se habia me                  | 2          | 17                |
|                 |               | si yo opino que  estaría bien frenamos un que | 3        |                   |
|                 |               | la ultima vez que estabas fue pensando en | 5          |                   |
|                 |               | hola como andan que la verdad es no      | 5          |                   |
|                 |               | saben si no sabía te contado iba         | 2          |                   |
</font>

<font size = 4>
<br>

_En la tabla 5 se presentan los resultados de coherencia obtenidos por el autocompletado del teclado de Google_
</font>

<font size=5>Tabla 5</font>

<font size = 2>

| Frase (final)                                | Coherencia | Coherencia total |
|---------------------------------------------|------------|-------------------|
| buenas tardes adjunto envío el archivo      | 3          | 17                |
| si yo opino que se si es posible            | 5          |                   |
| la ultima vez que se si es posible          | 4          |                   |
| hola como andan por allá y que no           | 2          |                   |
| saben si van a ser un poco                  | 3          |                   |
</font>

## 5. Reentrenamiento del modelo
Ajustamos el código para que luego de cada frase terminada añadiera estos mensajes a la lista de entrenamiento y volviera a calcular las probabilidades condicionales y a priori. Lo incluímos como un loop adicional y no como el principal debido al largo tiempo de procesamiento que lleva utilizarlo para replicar las pruebas.

*   Desafiamos al modelo probando la frase "_australopitecus aguaraguazú intrépidamente cabilan_". Inicialmente las sugerencias fueron incoherentes, generalmente con una palabra muy común (por ejemplo "que") y de baja confiabilidad (aprox. 0,0219). Luego de ingresar la frase correcta, si bien involucra un tiempo considerable de cómputo, el modelo respondió correctamente al agregado de nueva información, esto se ve reflejado al ingresar nuevamente la palabra "_australopitecus_" y observando que las sugerencias eran coherentes (_intrépidamente_) y con una confiabilidad mucho mayor (aprox. 0,166). Esto nos muestra la capacidad del modelo de adaptarse a nueva información. 

## 6. Conclusiones

* **Coherencia de Sugerencias**: Nuestros resultados en términos de coherencia de las sugerencias son prometedores. Se alcanzaron niveles de coherencia comparables a sistemas de recomendación más avanzados, como el teclado de Google.

* **Rendimiento y Tiempo de Respuesta**: Sin embargo, en lo que respecta al tiempo que demora en sugerir la siguiente palabra, observamos que es demasiado largo para aplicaciones en un escenario real. Este es un aspecto que requiere mejoras significativas para hacer que el modelo sea práctico en situaciones cotidianas.

* **Mejor Conjunto de Parámetros**: En nuestras pruebas, el conjunto de parámetros que mejor funcionó fue aquel con un m igual a 2, horizonte de 4 y filtrado por diccionario. Aunque su confiabilidad no es la más alta, es comparable con otros conjuntos de parámetros. Damos prioridad a la coherencia subjetiva, ya que consideramos que es la métrica más relevante en un uso real. Además, el filtrado con diccionario logró reducir la cantidad de palabras sugeridas y mejoró significativamente el tiempo de respuesta en comparación con el mismo modelo sin filtrado. Sin embargo, se debe tener en cuenta que este enfoque puede eliminar palabras relevantes, como nombres propios.

* **Limitaciones del Entrenamiento**: Es importante destacar que logramos resultados de coherencia bastante buenos, pero esto requirió una cantidad considerable de texto de entrenamiento. Esto plantea una limitación importante en términos del tiempo necesario para entrenar estos modelos y debe ser considerado en su implementación práctica.

En resumen, nuestro estudio demuestra que el algoritmo de Naive Bayes tiene potencial para la clasificación de texto coherente, pero se deben abordar desafíos de rendimiento y entrenamiento para hacerlo más efectivo en aplicaciones del mundo real.